# First Predict Customers, then Predict Sales

In [1]:
import pandas as pd
import numpy as np

# Import Data

In [2]:
train = pd.read_csv('../cleaneddata/cleanedTraining.csv', index_col=0)
train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,LogSales,DayOfWeek,Store,WeekNumber,Customers
0,0,1,1,0,0,0,0,1,0,0,1,0,1,0,0,8.568456,5,1,31,555
1,0,1,1,0,0,0,0,1,0,0,1,0,1,0,0,8.521185,4,1,31,546
2,0,1,1,0,0,0,0,1,0,0,1,0,1,0,0,8.472614,3,1,31,523
3,0,1,1,0,0,0,0,1,0,0,1,0,1,0,0,8.519391,2,1,31,560
4,0,1,1,0,0,0,0,1,0,0,1,0,1,0,0,8.716372,1,1,31,612


## Train/Test Split

In [3]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train.drop('LogSales', axis=1), 
                                                    train.ix[:, ['LogSales', 'Customers']], 
                                                    random_state=123)

## Fit a Tree to Predict Customers

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.svm import SVR

cust_rf = RandomForestRegressor().fit(X_train.drop('Customers', axis=1), y_train['Customers'])
# cust_svr = SVR().fit(X_train.drop('Customers', axis=1), y_train['Customers'])

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

cust_pred = cust_rf.predict(X_test.drop('Customers', axis=1))
sqrt(mean_squared_error(cust_pred, y_test['Customers']))

## Fit a Tree to Take Predicted Custs into Account

In [57]:
logsales_rf = AdaBoostRegressor(RandomForestRegressor(), n_estimators=50).fit(train.drop('LogSales', axis=1), train['LogSales'])

KeyboardInterrupt: 

In [ ]:
logsales_pred = logsales_rf.predict(X_test)

In [ ]:
mean_squared_error(logsales_pred, y_test['LogSales'])

## Bring in Evaluation Set

In [21]:
test = pd.read_csv('../cleaneddata/cleanedtest.csv', index_col=0)
test.head()
# test.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,DayOfWeek,Store,WeekNumber,Open,Id
0,0,1,1,0,0,0,1,0,0,0,1,0,1,0,0,4,1,38,1,1
1,0,1,1,0,0,0,1,0,0,0,1,0,1,0,0,3,1,38,1,857
2,0,1,1,0,0,0,1,0,0,0,1,0,1,0,0,2,1,38,1,1713
3,0,1,1,0,0,0,1,0,0,0,1,0,1,0,0,1,1,38,1,2569
4,1,0,1,0,0,0,1,0,0,0,1,0,1,0,0,7,1,37,0,3425


In [23]:
test['Customers'] = cust_rf.predict(test.drop(['Open', 'Id'], axis=1))

In [24]:
test['predictedSales'] = logsales_rf.predict(test.drop(['Open', 'Id'], axis=1))

In [34]:
test['Sales'] = np.exp(test['predictedSales'])
test.ix[test['Open']==0,'Sales'] = 0

In [35]:
test.ix[:, ['Id', 'Sales']].to_csv('twoStageRandomForest.csv', index=False)